<a href="https://colab.research.google.com/github/nauqcreen/Optimization/blob/main/Optimization_covid19.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### **Phase 1: Mount Google Drive and install libraries**

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
print("PIP INSTALLING...")
!pip install pulp pandas openpyxl
print("PIP INSTALLED")

PIP INSTALLING...
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 16.4/16.4 MB 82.4 MB/s eta 0:00:00
PIP INSTALLED


Load data from Excel sheets

### **Phase 2: Load and preprocess data from Excel**

Load data from Excel

In [4]:
import pandas as pd

file_path = '/content/drive/MyDrive/Optimization_covid19/Optimization_LN2.xlsx'

try:
    df_demand = pd.read_excel(file_path, sheet_name='Demand_P_ij')
    df_weights = pd.read_excel(file_path, sheet_name='Priority_Weights')
    df_airplanes = pd.read_excel(file_path, sheet_name='Airplanes_Capacity')
    df_quarantine = pd.read_excel(file_path, sheet_name='Quarantine_Capacity')
    print("Successfully loaded all sheets from the Excel file.")
except FileNotFoundError:
    print(f"ERROR: File not found at '{file_path}'")
    raise
except ValueError as e:
    print(f"ERROR: Could not read one or more sheets. Please check sheet names in your Excel file.")
    print(f"Details: {e}")
    raise

Successfully loaded all sheets from the Excel file.


Preprocess and structure the data

In [5]:
# Clean 'City_ID' and 'PriorityGroup_ID'
df_demand['City_ID'] = df_demand['City_ID'].astype(str).str.strip()
df_demand['PriorityGroup_ID'] = df_demand['PriorityGroup_ID'].astype(str).str.strip()
df_weights['PriorityGroup_ID'] = df_weights['PriorityGroup_ID'].astype(str).str.strip()
df_airplanes['Airplane_ID'] = df_airplanes['Airplane_ID'].astype(str).str.strip()

1. Define Sets: CITIES, PRIORITY_GROUPS, AIRPLANES

In [6]:
CITIES = sorted(list(df_demand['City_ID'].unique()))
# Ensure PRIORITY_GROUPS are taken from df_weights
PRIORITY_GROUPS = sorted(list(df_weights['PriorityGroup_ID'].unique()))
AIRPLANES = sorted(list(df_airplanes['Airplane_ID'].unique()))

print("\n--- Sets ---")
print(f"CITIES (M): {CITIES}")
print(f"PRIORITY_GROUPS (N): {PRIORITY_GROUPS}")
print(f"AIRPLANES (U): {AIRPLANES}")


--- Sets ---
CITIES (M): ['C001', 'C002', 'C003', 'C004', 'C005']
PRIORITY_GROUPS (N): ['N1', 'N2', 'N3', 'N4']
AIRPLANES (U): ['QH001', 'QH002', 'VJ001', 'VJ002', 'VJ003', 'VN001', 'VN002', 'VN003']


2. Define Parameters

In [7]:
# P_ij: Demand from city i for priority group j
P_ij = {}
for index, row in df_demand.iterrows():
    P_ij[(row['City_ID'], row['PriorityGroup_ID'])] = row['Demand_P_ij']

# omega_j: Priority weight for group j
omega_j = {}
for index, row in df_weights.iterrows():
    omega_j[row['PriorityGroup_ID']] = row['Weight_omega_j']

# C_k: Capacity of airplane k
C_k = {}
for index, row in df_airplanes.iterrows():
    C_k[row['Airplane_ID']] = row['Capacity_C_k']

In [8]:
# QC: Total quarantine capacity
try:
    QC = df_quarantine[df_quarantine['Parameter'] == 'QuarantineCapacity_QC']['Value'].iloc[0]
except IndexError:
    print("ERROR: 'QuarantineCapacity_QC' not found or 'Value' column is missing in 'Quarantine_Capacity' sheet.")
    print("Please ensure the sheet has a row with 'Parameter' as 'QuarantineCapacity_QC' and a corresponding 'Value'.")
    raise
except KeyError as e:
    print(f"ERROR: Column {e} not found in 'Quarantine_Capacity' sheet. Expected columns: 'Parameter', 'Value'.")
    raise

In [9]:
# Detailed output
city_names_map = pd.Series(df_demand.City_Name.values, index=df_demand.City_ID).to_dict()
priority_group_names_map = pd.Series(df_weights.PriorityGroup_Name.values, index=df_weights.PriorityGroup_ID).to_dict()
airplane_details_map = {row['Airplane_ID']: {'Type': row['Airplane_Type'], 'Airline': row['Airline']} for index, row in df_airplanes.iterrows()}

In [10]:
print("\n--- Parameters ---")
# Some instances
if CITIES and PRIORITY_GROUPS:
    example_city = CITIES[0]
    example_group = PRIORITY_GROUPS[0]
    print(f"P_ij for ({example_city}, {example_group}): {P_ij.get((example_city, example_group), 'Not Defined')}")
if PRIORITY_GROUPS:
    example_group = PRIORITY_GROUPS[0]
    print(f"omega_j for {example_group}: {omega_j.get(example_group, 'Not Defined')}")
if AIRPLANES:
    example_airplane = AIRPLANES[0]
    print(f"C_k for {example_airplane}: {C_k.get(example_airplane, 'Not Defined')}")
print(f"QC: {QC}")

V_BIG_M = 1000000

print("\nStep 2 Completed: Data loaded and preprocessed!")


--- Parameters ---
P_ij for (C001, N1): 7500
omega_j for N1: 10
C_k for QH001: 290
QC: 5000

Step 2 Completed: Data loaded and preprocessed!


### **Phase 3: Define MILP model - problem, variables, and objective function**

3. Initialize the MILP Problem

In [11]:
from pulp import LpProblem, LpMaximize, LpVariable, lpSum, LpBinary, LpInteger

prob = LpProblem("RepatriationSchedulingProblem", LpMaximize)
print("Initialized LpProblem")

Initialized LpProblem


4. Define decision variables

In [12]:
# x_ik: Binary variable, 1 if airplane k is assigned to city i, 0 otherwise
# LpVariable.dicts(name, (index_set_1, index_set_2, ...), cat=...)
x_ik = LpVariable.dicts("x_ik", (CITIES, AIRPLANES), cat=LpBinary)
print("Defined x_ik variables")

Defined x_ik variables


In [13]:
# R_ij: Integer variable, number of citizens of group j from city i repatriated
# lowBound is 0. upBound is the initial demand P_ij for that group in that city.
R_ij = LpVariable.dicts("R_ij", (CITIES, PRIORITY_GROUPS), lowBound=0, cat=LpInteger)
for i in CITIES:
    for j in PRIORITY_GROUPS:
        R_ij[i][j].upBound = P_ij.get((i, j), 0)
print("Defined R_ij variables and set their upper bounds")

Defined R_ij variables and set their upper bounds


In [14]:
# L_i: Integer variable, total number of citizens repatriated from city i
L_i = LpVariable.dicts("L_i", CITIES, lowBound=0, cat=LpInteger)
print("Defined L_i variables")

Defined L_i variables


5. Define auxiliary variables (as per Al-Shihabi & Mladenović, 2022)

In [15]:
# P_cum_ij: Cumulative demand up to priority group j in city i (PARAMETER, calculated from P_ij)
# The PRIORITY_GROUPS list was sorted alphabetically. Need a ver sorted by omega_j
sorted_priority_groups_by_weight = sorted(PRIORITY_GROUPS, key=lambda pg_key: omega_j[pg_key], reverse=True)

P_cum_ij = {}
for i in CITIES:
    current_sum = 0
    for j_group in sorted_priority_groups_by_weight:
        current_sum += P_ij.get((i, j_group), 0)
        P_cum_ij[(i, j_group)] = current_sum
print("Calculated P_cum_ij parameter")

# P_hat_cum_ij: Variable, cumulative number of people in group j and higher priority groups
# AFTER L_i citizens are repatriated from city i
P_hat_cum_ij = LpVariable.dicts("P_hat_cum_ij", (CITIES, PRIORITY_GROUPS), lowBound=0, cat=LpInteger)
print("Defined P_hat_cum_ij variables")

# epsilon_i: Binary variable, auxiliary for L_i calculation
epsilon_i = LpVariable.dicts("epsilon_i", CITIES, cat=LpBinary)
print("Defined epsilon_i variables")

# alpha_ij: Binary variable, auxiliary for P_hat_cum_ij calculation
alpha_ij = LpVariable.dicts("alpha_ij", (CITIES, PRIORITY_GROUPS), cat=LpBinary)
print("Defined alpha_ij variables")

Calculated P_cum_ij parameter
Defined P_hat_cum_ij variables
Defined epsilon_i variables
Defined alpha_ij variables


6. Objective function

In [16]:
# Maximize sum(omega_j * R_ij) for all i in CITIES and j in PRIORITY_GROUPS
prob += lpSum(omega_j[j] * R_ij[i][j] for i in CITIES for j in PRIORITY_GROUPS), "Maximize_Total_Weighted_Priority_Score"
print("Defined OF")

print("\nStep 3 Completed: MILP problem initialized, variables defined, and OF are set")

Defined OF

Step 3 Completed: MILP problem initialized, variables defined, and OF are set


### **Phase 4: Defining model constraints**

In [17]:
# Constraint: L_i = sum over j (R_ij[i][j])
for i in CITIES:
    prob += L_i[i] == lpSum(R_ij[i][j] for j in PRIORITY_GROUPS), f"Constraint_TotalRepatriatedFromCity_{i}"
print("  - Defined constraints for L_i (total from city)")

  - Defined constraints for L_i (total from city)


In [18]:
# Constraints for L_i based on airplane capacity and total demand in the city
# P_total_i calculates the total demand in city i across all priority groups
P_total_i = {city_id: P_cum_ij.get((city_id, sorted_priority_groups_by_weight[-1]), 0) for city_id in CITIES}

for i in CITIES:
    # Equations 3 & 4: Define epsilon_i
    # epsilon_i = 1 if P_total_i[i] > sum(C_k[k_plane] * x_ik[i][k_plane]), 0 otherwise
    prob += P_total_i[i] - lpSum(C_k[k_plane] * x_ik[i][k_plane] for k_plane in AIRPLANES) <= epsilon_i[i] * V_BIG_M, f"Constraint_Epsilon1_{i}"
    prob += P_total_i[i] - lpSum(C_k[k_plane] * x_ik[i][k_plane] for k_plane in AIRPLANES) >= (epsilon_i[i] - 1) * V_BIG_M + 0.001, f"Constraint_Epsilon2_{i}"

    # Equations 5 & 6: If epsilon_i = 0 (demand <= assigned capacity), then L_i = P_total_i[i]
    # If epsilon_i = 0 (P_total_i <= sum_assigned_capacity), L_i = P_total_i
    prob += L_i[i] <= P_total_i[i] + epsilon_i[i] * V_BIG_M, f"Constraint_L_i_if_epsilon0_upper_{i}"
    prob += L_i[i] >= P_total_i[i] - epsilon_i[i] * V_BIG_M, f"Constraint_L_i_if_epsilon0_lower_{i}"

    # Equations 7 & 8: If epsilon_i = 1 (demand > assigned capacity), then L_i = sum_assigned_capacity
    prob += L_i[i] <= lpSum(C_k[k_plane] * x_ik[i][k_plane] for k_plane in AIRPLANES) + (1 - epsilon_i[i]) * V_BIG_M, f"Constraint_L_i_if_epsilon1_upper_{i}"
    prob += L_i[i] >= lpSum(C_k[k_plane] * x_ik[i][k_plane] for k_plane in AIRPLANES) - (1 - epsilon_i[i]) * V_BIG_M, f"Constraint_L_i_if_epsilon1_lower_{i}"
print("  - Defined constraints for L_i based on epsilon_i (airplane capacity vs demand).")

  - Defined constraints for L_i based on epsilon_i (airplane capacity vs demand).


In [19]:
# Constraints for P_hat_cum_ij
for i in CITIES:
    for j_group in sorted_priority_groups_by_weight:
        # Equations 9 & 10: Define alpha_ij[i][j_group]
        # alpha_ij = 1 if L_i[i] >= P_cum_ij[(i,j_group)], 0 otherwise
        prob += L_i[i] - P_cum_ij.get((i, j_group), 0) <= alpha_ij[i][j_group] * V_BIG_M, f"Constraint_Alpha1_{i}_{j_group}"
        prob += L_i[i] - P_cum_ij.get((i, j_group), 0) >= (alpha_ij[i][j_group] - 1) * V_BIG_M + 0.001, f"Constraint_Alpha2_{i}_{j_group}"

        # Equations 11 & 12: If alpha_ij[i][j_group] = 1 (L_i repatriates all in this cumulative group), P_hat_cum_ij = 0
        prob += P_hat_cum_ij[i][j_group] <= (1 - alpha_ij[i][j_group]) * V_BIG_M, f"Constraint_P_hat_if_alpha1_upper_{i}_{j_group}"
        # P_hat_cum_ij is non-negative by definition (lowBound=0)
        prob += P_hat_cum_ij[i][j_group] >= - (1 - alpha_ij[i][j_group]) * V_BIG_M, f"Constraint_P_hat_if_alpha1_lower_dummy_{i}_{j_group}"


        # Equations 13 & 14: If alpha_ij[i][j_group] = 0 (L_i does not repatriate all), P_hat_cum_ij = P_cum_ij - L_i
        prob += P_hat_cum_ij[i][j_group] <= P_cum_ij.get((i, j_group), 0) - L_i[i] + alpha_ij[i][j_group] * V_BIG_M, f"Constraint_P_hat_if_alpha0_upper_{i}_{j_group}"
        prob += P_hat_cum_ij[i][j_group] >= P_cum_ij.get((i, j_group), 0) - L_i[i] - alpha_ij[i][j_group] * V_BIG_M, f"Constraint_P_hat_if_alpha0_lower_{i}_{j_group}"
print("  - Defined constraints for P_hat_cum_ij (remaining cumulative demand) using alpha_ij.")

  - Defined constraints for P_hat_cum_ij (remaining cumulative demand) using alpha_ij.


In [20]:
# Constraint: Calculating R_ij (actual repatriated from group j, city i)
for j_idx, j_group in enumerate(sorted_priority_groups_by_weight):
        if j_idx == 0: # Highest priority group
            prob += R_ij[i][j_group] == P_cum_ij.get((i, j_group), 0) - P_hat_cum_ij[i][j_group], f"Constraint_Calc_R_ij_first_group_{i}_{j_group}"
        else:
            j_prev = sorted_priority_groups_by_weight[j_idx-1]
            prob += R_ij[i][j_group] == (P_cum_ij.get((i, j_group), 0) - P_hat_cum_ij[i][j_group]) - \
                                       (P_cum_ij.get((i, j_prev), 0) - P_hat_cum_ij[i][j_prev]), f"Constraint_Calc_R_ij_other_groups_{i}_{j_group}"
print("  - Defined constraints for R_ij (actual repatriated per group).")

  - Defined constraints for R_ij (actual repatriated per group).


In [21]:
# Constraint: Total repatriated citizens cannot exceed quarantine capacity
prob += lpSum(L_i[i] for i in CITIES) <= QC, "Constraint_QuarantineCapacity"
print("  - Defined Quarantine Capacity constraint")

  - Defined Quarantine Capacity constraint


In [22]:
# Constraint: Each airplane can be assigned to at most one city
for k_plane in AIRPLANES:
    prob += lpSum(x_ik[i][k_plane] for i in CITIES) <= 1, f"Constraint_AirplaneAssignment_{k_plane}"
print("  - Defined Airplane Assignment constraints")

  - Defined Airplane Assignment constraints


In [23]:
# Sanity Constraint: L_i cannot exceed total demand P_total_i[i]
for i in CITIES:
    prob += L_i[i] <= P_total_i[i], f"Constraint_Li_lessthan_TotalDemandInCity_{i}"
print("  - Defined L_i upper bound by total city demand.")

print("\nStep 4 Completed: All model constraints defined")

  - Defined L_i upper bound by total city demand.

Step 4 Completed: All model constraints defined


### **Phase 5: Solve the MILP problem**

In [24]:
prob.solve()

solution_status = pulp.LpStatus[prob.status]
print(f"\n--- Solution status ---")
print(f"Status: {solution_status}")

# Optimal value
if prob.objective is not None and prob.status == pulp.LpStatusOptimal:
    objective_value = pulp.value(prob.objective)
    print(f"Total weighted priority score: {objective_value:.2f}")
else:
    print("Objective value not available")

NameError: name 'pulp' is not defined

import pulp



In [25]:
import pulp

prob.solve()

solution_status = pulp.LpStatus[prob.status]
print(f"\n--- Solution status ---")
print(f"Status: {solution_status}")

# Optimal value
if prob.objective is not None and prob.status == pulp.LpStatusOptimal:
    objective_value = pulp.value(prob.objective)
    print(f"Total weighted priority score: {objective_value:.2f}")
else:
    print("Objective value not available")


--- Solution status ---
Status: Optimal
Total weighted priority score: 22110.00


In [27]:
if prob.status == pulp.LpStatusOptimal or prob.status == pulp.LpStatusFeasible:
    print("\n--- REPATRIATION SCHEDULE ---")
    total_repatriated_overall = 0
    any_flight_scheduled = False

    for i in CITIES:
        city_name_display = city_names_map.get(i, i)
        total_repatriated_from_city_i = pulp.value(L_i[i])
        if total_repatriated_from_city_i is None: total_repatriated_from_city_i = 0

        total_repatriated_overall += total_repatriated_from_city_i
        print(f"\nCity: {city_name_display} (ID: {i})")

        assigned_planes_details = []
        city_had_flights = False
        for k_plane in AIRPLANES:
            if x_ik[i][k_plane].varValue is not None and pulp.value(x_ik[i][k_plane]) > 0.5:
                plane_detail = airplane_details_map.get(k_plane, {'Airline': 'N/A', 'Type': 'N/A'})
                assigned_planes_details.append(
                    f"  - Airplane {k_plane} ({plane_detail['Airline']} - {plane_detail['Type']}, Capacity: {C_k[k_plane]})"
                )
                city_had_flights = True
                any_flight_scheduled = True

        if city_had_flights:
            print("  Assigned Airplanes:")
            for detail_str in assigned_planes_details:
                print(detail_str)
        else:
            print("  No airplanes assigned to this city.")

        print(f"  Total Repatriated from this city (L_{i}): {total_repatriated_from_city_i:.0f}")

        if total_repatriated_from_city_i > 0:
            print("  Repatriation by Priority Group:")
            for j_group in sorted_priority_groups_by_weight:
                repatriated_R_ij = pulp.value(R_ij[i][j_group])
                if repatriated_R_ij is None: repatriated_R_ij = 0

                if repatriated_R_ij > 0.01: # Show only if >0 to avoid clutter
                    group_name_display = priority_group_names_map.get(j_group, j_group)
                    initial_demand_P_ij = P_ij.get((i, j_group), 0)
                    print(f"    - Group {j_group} ({group_name_display}): {repatriated_R_ij:.0f} (Initial Demand: {initial_demand_P_ij})")

    if not any_flight_scheduled and total_repatriated_overall == 0 :
         print("\nNo flights were scheduled, and no citizens were repatriated in this plan.")

    print("\n\n\n")
    print("\n--- OVERALL STATISTICS ---")
    print(f"Total citizens repatriated across all cities: {total_repatriated_overall:.0f}")
    print(f"Quarantine capacity used: {total_repatriated_overall:.0f} / {QC}")
    print(f"Percentage of quarantine capacity used: {total_repatriated_overall / QC * 100:.2f}%")
    print(f"Total weighted priority score: {objective_value:.2f}")
    print(f"Percentage of total priority score: {objective_value / total_repatriated_overall * 100:.2f}%")
    print(f"Total number of flights scheduled: {any_flight_scheduled}")
    print(f"Total number of airplanes assigned: {len(AIRPLANES)}")
    print(f"Total number of cities considered: {len(CITIES)}")
    print(f"Total number of priority groups considered: {len(PRIORITY_GROUPS)}")
    print(f"Total number of airplane types considered: {len(AIRPLANES)}")
    print("\n\n\n")
    print("\n--- Auxiliary variables ---")
    for i in CITIES:
      print(f"City {i}: epsilon_i = {pulp.value(epsilon_i[i]):.0f}")
      for j_group in sorted_priority_groups_by_weight:
        if alpha_ij[i][j_group].varValue is not None:
          print(f"  Group {j_group}: alpha_ij = {pulp.value(alpha_ij[i][j_group]):.0f}, P_hat_cum = {pulp.value(P_hat_cum_ij[i][j_group]):.0f}")

elif prob.status == pulp.LpStatusInfeasible:
    print("The problem is INFEASIBLE")
elif prob.status == pulp.LpStatusUnbounded:
    print("The problem is UNBOUNDED")
else:
    print(f"Problem was not solved to optimality or feasibility. Status: {solution_status}")

print("\nStep 5 Solved")


--- REPATRIATION SCHEDULE ---

City: Tokyo (Nhật Bản) (ID: C001)
  No airplanes assigned to this city.
  Total Repatriated from this city (L_C001): 0

City: Seoul (Hàn Quốc) (ID: C002)
  Assigned Airplanes:
  - Airplane VJ001 (Vietjet Air - Airbus A330-300, Capacity: 370)
  - Airplane VJ002 (Vietjet Air - Airbus A321NEO/CEO, Capacity: 230)
  - Airplane VJ003 (Vietjet Air - Airbus A321NEO/CEO, Capacity: 230)
  - Airplane VN003 (Vietnam Airlines - Airbus A350-900, Capacity: 315)
  Total Repatriated from this city (L_C002): 1145
  Repatriation by Priority Group:
    - Group N1 (N1_KhanCap): 1145 (Initial Demand: 1500)

City: Taipei (Đài Loan) (ID: C003)
  Assigned Airplanes:
  - Airplane QH001 (Bamboo Airways - Boeing 787-9, Capacity: 290)
  - Airplane QH002 (Bamboo Airways - Airbus A321neo, Capacity: 196)
  - Airplane VN002 (Vietnam Airlines - Boeing 787-9, Capacity: 290)
  Total Repatriated from this city (L_C003): 776
  Repatriation by Priority Group:
    - Group N1 (N1_KhanCap): 776 